In [526]:
import numpy as np

In [527]:
rho_tank=4500 #kg/m3
m=0.28#kg/s
P=100#bar
T=273.15+25
Tref=10+273.15
Po=31

In [528]:
m

0.28

In [529]:
def Cp_Cv(T):
        return 1.5949974007561318 * T ** (-0.021034016066268504)
def compressor_power(m_dot, T1, P_ratio, gamma, cp, eta):
    """
    Calcula la potencia requerida por un compresor para la compresión adiabática de un gas.

    Parámetros:
      m_dot : float
          Caudal másico del gas (kg/s).
      T1 : float
          Temperatura de entrada al compresor (K).
      P_ratio : float
          Relación de presión (P2 / P1, sin unidades) a la que se desea comprimir.
      gamma : float
          Relación de calores específicos (Cp/Cv).
      cp : float
          Calor específico a presión constante (J/kg·K).
      eta : float
          Eficiencia del compresor (valor entre 0 y 1).

    Retorna:
      power : float
          Potencia del compresor requerida (W, vatios).
    """
    # Cálculo del trabajo específico en J/kg
    specific_work = (cp * T1 * gamma / (gamma - 1)) * ((P_ratio)**((gamma - 1) / gamma) - 1)
    
    # Potencia requerida = caudal másico * trabajo específico, corregido por la eficiencia
    power = m_dot * specific_work / eta
    return power/1000

# Parámetros de entrada (se pueden ajustar según el caso particular)
T1      = T          # Temperatura de entrada en Kelvin
P_ratio = P1/P2           # Relación de presiones (por ejemplo, de 1 a 10)
gamma   = Cp_Cv(T)          # Relación de calores específicos (puede variar para hidrógeno, p.ej., ~1.40 en bajas temperaturas)
CP=19.67099783 + 0.069681519*T - 0.000200098*T**2 + 2.89493E-07*T**3 - 2.22475E-10*T**4 + 8.81466E-14*T**5 - 1.42043E-17*T**6
CP=CP/2*1000
cp      = CP        # Calor específico de hidrógeno a presión constante (J/kg·K). Nota: El valor de hidrógeno es mayor debido a su baja masa molar.
eta     = 0.75          # Eficiencia del compresor (75%)

CP=19.67099783 + 0.069681519*T - 0.000200098*T**2 + 2.89493E-07*T**3 - 2.22475E-10*T**4 + 8.81466E-14*T**5 - 1.42043E-17*T**6
CP=CP/2*1000
    
    
power_required = compressor_power(m, T, P_ratio, Cp_Cv(T), CP, eta)
power_required

np.float64(-488.1852572334739)

In [530]:
import pandas as pd
crit=pd.DataFrame()
crit['Comp']=['H2']
crit['Tc']=[32.938]
crit['Pc']=[1293.00]
crit['w']=[-0.2160]
crit['PM']=[1.00784*2]
T=298
P1=100
def PRv2(Tc,Pc,mm,w,T,P):
    #Unidades: P, Pc: kPa, T, Tc: K, w: Adim, mm: kg/kmol
    R=8.314#J/mol-K
    Ru=R/mm
    mi=0.379642+1.48503*w-0.164423*w**2+0.016666*w**3
    alpha=(1+mi*(1-(T/Tc)**0.5))**2
    a=0.45724*((Ru)**2)*(Tc**2)*alpha/(Pc)
    b=0.0778*(Ru)*(Tc/Pc)
    A_may=a*P/(((Ru)**2)*(T**2))
    B_may=b*P/((Ru)*T)
    m=-(1-B_may)
    n=(A_may-(3*B_may**2)-(2*B_may))
    o=-((A_may*B_may)-(B_may**2)-(B_may**3))
    v=(Ru*T)/P
    return {'alpha':alpha,'a':a,'b':b,'A':A_may,'B':B_may,'m':m,'n':n,'o':o,'v':v}
resul_peng=PRv2(crit['Tc'][0],crit['Pc'][0],crit['PM'][0],crit['w'][0],316+273.15,172.37*100)
z=np.real(max(np.roots([1,resul_peng['m'],resul_peng['n'],resul_peng['o']])))
V=(z*8.314*T)/(P1*1e6)
V

np.float64(2.594418378271237e-05)

In [531]:
import numpy as np
from scipy.optimize import root
def trabajo_reversible(m, P1, P2, eta, T2):
    P1=P1*100
    P2=P2*100
    
    resul_peng=PRv2(crit['Tc'][0],crit['Pc'][0],crit['PM'][0],crit['w'][0],T1,P1)
    z=np.real(max(np.roots([1,resul_peng['m'],resul_peng['n'],resul_peng['o']])))
    r1=(P1*crit['PM'][0])/(z*(8.314)*T1)
    
    V1=1/r1
    resul_peng=PRv2(crit['Tc'][0],crit['Pc'][0],crit['PM'][0],crit['w'][0],T2+273.15,P2)
    z=np.real(max(np.roots([1,resul_peng['m'],resul_peng['n'],resul_peng['o']])))
    r2=(P2*crit['PM'][0])/(z*(8.314)*(T2+273.15))
    V2=1/r2
    n = np.log(P2/P1)/(np.log((V1)/(V2)))
    
    term1 = (m * P1 * 1000 * V1) / (0.72*(n - 1)/n)
    term2 = (P2 / P1) ** ((n - 1) / n) - 1
    W_rev=term1*term2/1000
    return W_rev/eta

def trabajo_reversible_IT(tt, m, P1, P2, eta):
    P1=P1*100
    P2=P2*100
    T2=abs(tt[0])
    
    resul_peng=PRv2(crit['Tc'][0],crit['Pc'][0],crit['PM'][0],crit['w'][0],T1,P1)
    z=np.real(max(np.roots([1,resul_peng['m'],resul_peng['n'],resul_peng['o']])))
    r1=(P1*crit['PM'][0])/(z*(8.314)*T1)
    
    V1=1/r1
    resul_peng=PRv2(crit['Tc'][0],crit['Pc'][0],crit['PM'][0],crit['w'][0],T2,P2)
    z=np.real(max(np.roots([1,resul_peng['m'],resul_peng['n'],resul_peng['o']])))
    r2=(P2*crit['PM'][0])/(z*(8.314)*(T2))
    V2=1/r2
    n = np.log(P2/P1)/(np.log((V1)/(V2)))
    
    term1 = (1 * P1 * V1 * 1000) / (0.72*(n - 1)/n)
    term2 = (P2 / P1) ** ((n - 1) / n) - 1
    Tref=T1
    T=T2
    HhH2=(19.67099783*(T-Tref) + 0.069681519/2*(T**2-Tref**2) - 0.000200098/3*(T**3-Tref**3) + 2.89493E-07/4*(T**4-Tref*4) - 2.22475E-10/5*(T**5-Tref**5) + 8.81466E-14/6*(T**6-Tref**6) - 1.42043E-17/7*(T**6-Tref**6))/2
    HhH2=HhH2
    T=T1
    HhH1=(19.67099783*(T-Tref) + 0.069681519/2*(T**2-Tref**2) - 0.000200098/3*(T**3-Tref**3) + 2.89493E-07/4*(T**4-Tref*4) - 2.22475E-10/5*(T**5-Tref**5) + 8.81466E-14/6*(T**6-Tref**6) - 1.42043E-17/7*(T**6-Tref**6))/2
    HhH1=HhH1
    W_rev=term1*term2/1000
    print('kW',W_rev)
    return W_rev - (HhH2 - HhH1)

P1=31
P2=130
m=1.2
eta=0.75
sol = root(trabajo_reversible_IT, x0=500, args=(m, P1, P2, eta), method='hybr')
if sol.success:
    print(sol.message)
else:
    print(sol.message)

resultado = trabajo_reversible(m, P1, P2, eta, sol.x[0]-273.15)
print(f"W: {resultado:.20f} kW")

kW 3290.179691544105
kW 3290.179691544105
kW 3290.179691544105
kW 3290.179717593273
kW 3394.917784444857
kW 3394.3921512282363
kW 3394.3948351765303
kW 3394.3948352475045
The solution converged.
W: 5431.03173639600845490349 kW


In [532]:
import numpy as np
from scipy.optimize import root

def balance(x, T, Tref, m):
    global e
    global s
    """
    Calcula la función de balance para el proceso,
    devolviendo la diferencia e - s.

    Parámetros:
      x    : vector de incógnitas; se asume que x[0] es la variable a encontrar.
      T    : temperatura en K.
      Tref : temperatura de referencia en K.
      m    : masa del hidrógeno (kg).

    La función utiliza dos series polinómicas (para H2 y H2O)
    para calcular, respectivamente, las energías 'e' y 's', y retorna e - s.
    """
    # Calcular la entalpía (u otra propiedad) para hidrógeno
    Ta=15+273.15
    HhH2 = ( 19.67099783*(T - Tref)
              + (0.069681519/2) * (T**2 - Tref**2)
              - (0.000200098/3) * (T**3 - Tref**3)
              + (2.89493E-07/4) * (T**4 - Tref**4)
              - (2.22475E-10/5) * (T**5 - Tref**5)
              + (8.81466E-14/6) * (T**6 - Tref**6)
              - (1.42043E-17/7) * (T**6 - Tref**6)
            ) / 2
    e = HhH2 * m

    # Calcular la entalpía (u otra propiedad) para agua (H2O)
    HhH2O = ( -22.41701677*(Ta - Tref)
               + (0.876972156/2) * (Ta**2 - Tref**2)
               - (0.002570393/3) * (Ta**3 - Tref**3)
               + (2.48383E-06/4) * (Ta**4 - Tref**4)
             ) / 18.015
    s = HhH2O * x[0]
    
    return e - s


In [533]:
Po

31

In [534]:
P=172.4
Po=31
P_diff=P-Po
P1=Po
P2=P
N_etapas=np.ceil((np.log(P2)/np.log(P1))/np.log(2))
N_etapas
P_etapa=P_diff/N_etapas
print(P_etapa)
Po=Po
Pf=P
P_acum=[Po]
Temps=[298]
Tref=T1
m=0.85
eta=0.75
Qs,Ts,m_H2O,comp,power,Ps=[],[],[],[],[],[]
for i in range(int(N_etapas)):
    P1=sum(P_acum)
    print(P1)
    P_acum.append(P_etapa)
    P2=sum(P_acum)
    Ti=(sum(P_acum)*V2*2)/(m*Z*8.314)
    print('T',Ti)
    # Valor inicial para la incógnita x (en este caso, es un vector de una dimensión)
    x0 = [1.0]
    sol = root(trabajo_reversible_IT, x0=500, args=(m, P1, P2, eta), method='hybr')
    if sol.success:
        print(sol.message)
    else:
        print(sol.message)

    resultado = trabajo_reversible(m, P1, P2, eta, sol.x[0]-273.15)
    print(f"W: {resultado:.20f} kW")
    # Se utiliza scipy.optimize.root con método 'hybr'
    solucion = root(balance, x0, args=(25+273.15, sol.x[0], m), method='hybr', tol=1e-1)

    if solucion.success:
        x_encontrado = solucion.x[0]
        print(f"Solución encontrada: x = {x_encontrado:.6f}")
        Temps.append(25+273.15)
        Qs.append(float(e))
        Ts.append((25+273.15))
        m_H2O.append(float(x_encontrado))
        comp.append('No. '+str(i+1))
        CP=19.67099783 + 0.069681519*T - 0.000200098*T**2 + 2.89493E-07*T**3 - 2.22475E-10*T**4 + 8.81466E-14*T**5 - 1.42043E-17*T**6
        CP=CP/2*1000
        power.append(resultado)
        Ps.append(P2)
    else:
        print("No se encontró solución:", solucion.message)
        

47.13333333333333
31
T 52.00727436806704
kW 2103.5403006874303
kW 2103.5403006874303
kW 2103.5403006874303
kW 2103.5403174973912
kW 1947.3773942692658
kW 1946.11875899042
kW 1946.1085722418463
kW 1946.1085715650456
The solution converged.
W: 2205.58971444038479603478 kW
Solución encontrada: x = 2.720415
78.13333333333333
T 83.38026325733614
kW 1091.5203751871707
kW 1091.5203751871707
kW 1091.5203751871707
kW 1091.5203838131429
kW 925.2316100034723
kW 923.2596209213932
kW 923.2337011454446
kW 923.2336962552089
kW 923.2336962551962
The solution converged.
W: 1046.33152242255550845584 kW
Solución encontrada: x = 2.513266
125.26666666666665
T 114.75325214660525
kW 751.2935697169793
kW 751.2935697169793
kW 751.2935697169793
kW 751.2935755802413
kW 619.9227245577546
kW 618.3414872342253
kW 618.3192202840117
kW 618.3192153614324
kW 618.3192153614166
The solution converged.
W: 700.76177740960554274352 kW
Solución encontrada: x = 2.350006


In [535]:
import pandas as pd
# Crear un DataFrame con los datos
data = {
	"Compressor": comp,
    "T (K)": Ts,
    "P (bar)": Ps,  # Convertir de Pa a bar
	"Cooling H2O (kg/s)": m_H2O,
	"Q (MW) Python": np.array(Qs)/1000,
    "Q (MW) Aspen":np.array([-1645.66707374,-775.158866276,-514.144266522])/1000,
    "W (MW) Python": np.array(power)/1000,
    "W (MW) Aspen":np.array([2211.5522, 1052.84936, 706.504967])/1000
    
}

df_compresores = pd.DataFrame(data)

df_compresores['Error %']=abs(((df_compresores['Q (MW) Aspen']-df_compresores['Q (MW) Python'])/df_compresores['Q (MW) Aspen']*100 + (df_compresores['W (MW) Aspen']-df_compresores['W (MW) Python'])/df_compresores['W (MW) Aspen']*100)/2)
df_compresores.loc[df_compresores.shape[0]] = ["Total", Temps[-1], Ps[-1], sum(df_compresores['Cooling H2O (kg/s)']), sum(df_compresores['Q (MW) Python']), sum(df_compresores['Q (MW) Aspen']), sum(df_compresores['W (MW) Python']), sum(df_compresores['W (MW) Aspen']), sum(df_compresores['Error %'])]
df_compresores

,Compressor,T (K),P (bar),Cooling H2O (kg/s),Q (MW) Python,Q (MW) Aspen,W (MW) Python,W (MW) Aspen,Error %
0,No. 1,298.15,78.133333,2.720415,-1.654192,-1.645667,2.205590,2.211552,0.124217
1,No. 2,298.15,125.266667,2.513266,-0.784749,-0.775159,1.046332,1.052849,0.309035
2,No. 3,298.15,172.400000,2.350006,-0.525571,-0.514144,0.700762,0.706505,0.704820
3,Total,298.15,172.400000,7.583686,-2.964512,-2.934970,3.952683,3.970907,1.138072


In [536]:
import pandas as pd
# Crear un DataFrame con los datos
data = {
	"Compressor": comp,
    "T (K)": Ts,
    "P (bar)": Ps,  # Convertir de Pa a bar
	"Cooling H2O (kg/s)": m_H2O,
	"Q (kW) Python": Qs,
    "Q (kW) Aspen":[-2636.1054600999996,-1039.633555013],
    "W (kW) Python": power,
    "W (kW) Aspen":[3546.3106, 1422.50884]
    
}

df_compresores = pd.DataFrame(data)
df_compresores.loc[df_compresores.shape[0]] = ["Total", Temps[-1], Ps[-1], sum(m_H2O), sum(Qs), sum([-2636.1054600999996,-1039.633555013]), sum(power), sum([3546.3106, 1422.50884])]
df_compresores['Error %']=abs(((df_compresores['Q (kW) Aspen']-df_compresores['Q (kW) Python'])/df_compresores['Q (kW) Aspen']*100 + (df_compresores['W (kW) Aspen']-df_compresores['W (kW) Python'])/df_compresores['W (kW) Aspen']*100)/2)
# Mostrar la tabla
df_compresores

ValueError: All arrays must be of the same length